# Session #2  Homework

This notebook represent the #2 Homework of ML-Zoomcamp 

### Dataset
In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

The goal of this homework is to create a regression model for predicting housing prices (column `'median_house_value'`).

### Features

For the rest of the homework, you'll need to use only these columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`

Select only them.

### Import packages

In [25]:
import pandas as pd
import numpy as np

### Get the data

In [26]:

url_data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
df = pd.read_csv(url_data)

### Inspect the data

In [27]:
df.sample(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20512,-121.55,38.59,36.0,435.0,95.0,285.0,90.0,1.2292,69600.0,INLAND
15568,-117.04,33.09,16.0,4677.0,581.0,1902.0,566.0,6.1834,335600.0,<1H OCEAN
4047,-118.51,34.14,28.0,6748.0,904.0,2431.0,876.0,12.8879,500001.0,<1H OCEAN
18127,-122.02,37.34,30.0,1036.0,151.0,467.0,156.0,6.4480,360600.0,<1H OCEAN
3548,-118.61,34.25,16.0,8295.0,1506.0,3903.0,1451.0,5.5111,276600.0,<1H OCEAN


Select only indicated columns

In [28]:
columns = ['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']
df = df[columns]

# Question 1
Find a feature with missing values. How many missing values does it have?

In [29]:
df.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

# Question 2
What's the median (50% percentile) for variable 'population'?

In [30]:
df['population'].median()

1166.0

# Regression

### Split the data

* Shuffle the initial dataset, use seed 42.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Make sure that the target value ('median_house_value') is not in your dataframe.
* Apply the log transformation to the median_house_value variable using the np.log1p() function.

Shuffle the initial dataset, use seed 42.

# Validation framework
Split your data in train/val/test sets, with 60%/20%/20% distribution.

In [31]:
np.random.seed(42)

n = len(df)

n_val = int(0.2 * n)
n_test = int(0.2 * n)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df.iloc[idx]

df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test = df_shuffled.iloc[n_train+n_val:].copy()

In [32]:
print(n,n_val+n_test+n_train )

20640 20640


Separate target variable ('median_house_value') and remove it 

In [33]:
target_variable = 'median_house_value'
y_train = df_train[target_variable]
y_val = df_val[target_variable]
y_test = df_test[target_variable]

del df_train[target_variable]
del df_val[target_variable]
del df_test[target_variable]

Apply the log transformation to the median_house_value variable using the np.log1p() function.

In [34]:
y_train = np.log1p(y_train.values)
y_val = np.log1p(y_val.values)
y_test = np.log1p(y_test.values)

# Question 3
* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using round(score, 2)
* Which option gives better RMSE?

In [35]:
def linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)


In [36]:
df_train.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
dtype: int64

Linear regression (Filling missing data with 0)

In [37]:
def prepare_X(df):
    X = df.copy()
    X = X.fillna(0).values
    return X

X_train = prepare_X(df_train)
w_0, w = linear_regression(X_train , y_train)


In [38]:
X_val = prepare_X(df_val)
y_pred = w_0 + X_val@w

score =rmse(y_val, y_pred)
round(score,2)

0.33

In [39]:
w_0, w 

(-11.686975241328462,
 array([-2.82087184e-01, -2.76255120e-01,  2.91329621e-03, -8.64531514e-06,
         1.50811922e-04, -2.08205659e-04,  5.69546573e-04,  1.71308139e-01]))

Linear regression (Filling missing data with mean)

In [40]:
   
def prepare_X(df,mean):
    X = df.copy()
    X = X.fillna(mean).values
    return X

total_bedrooms_mean = df_train['total_bedrooms'].mean()
X_train = prepare_X(df_train,total_bedrooms_mean)
w_0, w = linear_regression(X_train , y_train)


In [41]:
X_val = prepare_X(df_val,total_bedrooms_mean)
y_pred = w_0 + X_val@w

score =rmse(y_val, y_pred)
round(score,2)

0.33

R: The rmse score is equally good at filling in the missing values with 0 and with the mean.

### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0. 
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

In [42]:
def ridge_regression(X, y,r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [46]:
def prepare_X(df):
    X = df.copy()
    X = X.fillna(0).values
    return X

r_choices= [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10,100]

#prepare DF
X_train = prepare_X(df_train)
X_val = prepare_X(df_val)
for r in r_choices:
    #Train model    
    w_0, w = ridge_regression(X_train , y_train,r)
    #Prediction    
    y_pred = w_0 + X_val@w

    score =rmse(y_val, y_pred)
    print(f'the rmse score with r:{r} is',round(score,2))

the rmse score with r:0 is 0.33
the rmse score with r:1e-06 is 0.33
the rmse score with r:0.0001 is 0.33
the rmse score with r:0.001 is 0.33
the rmse score with r:0.01 is 0.33
the rmse score with r:0.1 is 0.33
the rmse score with r:1 is 0.33
the rmse score with r:5 is 0.34
the rmse score with r:10 is 0.34
the rmse score with r:100 is 0.34


### Question 5 

* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores. 
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

In [63]:
df

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,37.85,-122.24,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,37.85,-122.25,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,37.85,-122.25,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [67]:
def train_val_test(seed):
    np.random.seed(seed)

    n = len(df)

    n_val = int(0.2 * n)
    n_test = int(0.2 * n)
    n_train = n - (n_val + n_test)

    idx = np.arange(n)
    np.random.shuffle(idx)

    df_shuffled = df.iloc[idx]

    df_train = df_shuffled.iloc[:n_train].copy()
    df_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
    df_test = df_shuffled.iloc[n_train+n_val:].copy()
    
    target_variable = 'median_house_value'
    
    y_train = df_train[target_variable]
    y_val = df_val[target_variable]
    y_test = df_test[target_variable]
    
    del df_train[target_variable]
    del df_val[target_variable]
    del df_test[target_variable]
    
    y_train = np.log1p(y_train.values)
    y_val = np.log1p(y_val.values)
    y_test = np.log1p(y_test.values)
    
    
    return df_train, df_val, df_test, y_train, y_val, y_test

seed_choices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

scores =[]

for seed in seed_choices:
    df_train, df_val, df_test, y_train, y_val, y_test = train_val_test(seed)
    X_train = prepare_X(df_train)
    #Train model
    w_0, w = linear_regression(X_train , y_train)
    X_val = prepare_X(df_val)
    
    #Prediction
    y_pred = w_0 + X_val@w

    score =rmse(y_val, y_pred)
    score = round(score,2)
    scores.append(score)

In [68]:
np.array(scores).std()

0.005385164807134491

### Question 6

* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`. 
* What's the RMSE on the test dataset?

In [83]:
df_train, df_val, df_test, y_train, y_val, y_test = train_val_test(9)

df_train = pd.concat([df_train,df_val])
y_train = np.concatenate((y_train, y_val), axis=0)

X_train = prepare_X(df_train)
#Train model
w_0, w = ridge_regression(X_train , y_train,0.001)
X_test = prepare_X(df_test)

#Prediction
y_pred = w_0 + X_test@w

score =rmse(y_test, y_pred)
round(score,2)

0.35